In [9]:
import PySpin
import time
import sys

NUM_IMAGES = 12000  # Number of images to capture
TRIGGER_SOURCE = 'Line3'  # Trigger source to Line3 (green wire)

class TriggerType:
    HARDWARE = 2

CHOSEN_TRIGGER = TriggerType.HARDWARE

def configure_trigger(cam):
    """
    Configures the camera to use a hardware trigger on Line3.
    """
    try:
        # Ensure trigger mode off
        nodemap = cam.GetNodeMap()
        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
        node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())

        # Set TriggerSelector to FrameStart
        node_trigger_selector = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSelector'))
        node_trigger_selector_framestart = node_trigger_selector.GetEntryByName('FrameStart')
        node_trigger_selector.SetIntValue(node_trigger_selector_framestart.GetValue())

        # Select trigger source
        node_trigger_source = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSource'))
        node_trigger_source_hardware = node_trigger_source.GetEntryByName(TRIGGER_SOURCE)
        node_trigger_source.SetIntValue(node_trigger_source_hardware.GetValue())

        # Set TriggerActivation to RisingEdge
        node_trigger_activation = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerActivation'))
        node_trigger_activation_risingedge = node_trigger_activation.GetEntryByName('RisingEdge')
        node_trigger_activation.SetIntValue(node_trigger_activation_risingedge.GetValue())

        # Turn trigger mode on
        node_trigger_mode_on = node_trigger_mode.GetEntryByName('On')
        node_trigger_mode.SetIntValue(node_trigger_mode_on.GetValue())

        print('Trigger configured to hardware (Line3) with rising edge activation.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def grab_next_image_by_trigger(cam):
    """
    Waits for hardware trigger to capture image.
    """
    try:
        # Wait for hardware trigger to capture image
        print('Waiting for hardware trigger...')
        image_result = cam.GetNextImage()

        if image_result.IsIncomplete():
            print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
        else:
            print('Image grabbed successfully.')
            return image_result

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return None

    return None

def acquire_images(cam, nodemap, nodemap_tldevice):
    """
    Acquires and saves images from the camera.
    """
    try:
        result = True

        # Set acquisition mode to continuous
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())

        # Begin acquiring images
        cam.BeginAcquisition()
        print('Acquiring images...')

        # Retrieve device serial number for filename
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()

        # Retrieve, convert, and save images
        for i in range(NUM_IMAGES):
            image_result = grab_next_image_by_trigger(cam)
            if image_result:
                # Create a unique filename
                filename = f'Trigger-{device_serial_number}-{i}.jpg'

                # Save image
                image_result.Save(filename)
                print(f'Image saved at {filename}')

                # Release image
                image_result.Release()
            else:
                print('Failed to grab image.')

        # End acquisition
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result

def reset_trigger(nodemap):
    """
    Returns the camera to a normal state by turning off trigger mode.
    """
    try:
        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
        node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())
        print('Trigger mode disabled.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def print_device_info(nodemap):
    """
    Prints the device information of the camera.
    """
    print('*** DEVICE INFORMATION ***\n')
    try:
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
        if PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print(f'{node_feature.GetName()}: {node_feature.ToString() if PySpin.IsReadable(node_feature) else "Node not readable"}')
        else:
            print('Device control information not readable.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def run_single_camera(cam):
    """
    Runs the example on a single camera.
    """
    try:
        result = True

        # Retrieve TL device nodemap and print device information
        nodemap_tldevice = cam.GetTLDeviceNodeMap()
        result &= print_device_info(nodemap_tldevice)

        # Initialize camera
        cam.Init()

        # Retrieve GenICam nodemap
        nodemap = cam.GetNodeMap()

        # Configure trigger
        if not configure_trigger(cam):
            return False

        # Acquire images
        result &= acquire_images(cam, nodemap, nodemap_tldevice)

        # Reset trigger
        result &= reset_trigger(nodemap)

        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def main():
    """
    Main entry point for the example.
    """
    result = True

    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print(f'Library version: {version.major}.{version.minor}.{version.type}.{version.build}')

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()

    print(f'Number of cameras detected: {num_cameras}')

    # Finish if there are no cameras
    if num_cameras == 0:
        cam_list.Clear()
        system.ReleaseInstance()
        print('Not enough cameras!')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):
        print(f'Running example for camera {i}...')
        result &= run_single_camera(cam)
        print(f'Camera {i} example complete...')

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


Library version: 4.0.0.116
Number of cameras detected: 1
Running example for camera 0...
*** DEVICE INFORMATION ***

DeviceID: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000_0
DeviceSerialNumber: 16290112
DeviceUserID: 
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-13Y3M
DeviceVersion: 1808.0.120.0
DeviceBootloaderVersion: Node not readable
DeviceType: USB3Vision
DeviceDisplayName: FLIR Blackfly S BFS-U3-13Y3M
DeviceAccessStatus: OpenReadWrite
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000
DeviceLocation: 0000.0014.0000.021.000.000.000.000.000
DeviceCurrentSpeed: SuperSpeed
DeviceU3VProtocol: 1
DevicePortId: 21
GenICamXMLLocation: Device
GenICamXMLPath: 
GUIXMLLocation: Device
GUIXMLPath: Input.xml
Trigger configured to hardware (Line3) with rising edge activation.
Acquiring images...
Waiting for hardware trigger...
Image grabbed successfully.
Image saved at Trigger-16290112-0.jpg
Waiting for

SystemExit: 0